In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    message = client_socket.recv(1024).decode()
    print(f"Received message: {message}")

    client_socket.sendall(b"ACK: " + message.encode())

    with open("received_file.txt", 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data)
            print(f"Receiving file chunk...")

    print("File received successfully.")
    client_socket.close()


TCP Server is listening...
Connected to ('127.0.0.1', 49396)
Received message: yo
File received successfully.


In [ ]:
import socket

serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSocket.bind(('localhost', 65432))
serverSocket.listen(1)

print("TCP Server is listening...")

while True:
    clientSocket, clientAddress = serverSocket.accept()
    print(f"Connected to {clientAddress}")

    # Receive and process the text message
    message = clientSocket.recv(1024).decode()
    print(f"Received message: {message}")
    clientSocket.sendall(b"ACK: " + message.encode())

    # Receive file size first
    fileSize = int(clientSocket.recv(1024).decode())
    print(f"Expecting file of size: {fileSize} bytes")

    receivedSize = 0
    with open("received_file.txt", 'wb') as f:
        while receivedSize < fileSize:
            data = clientSocket.recv(1024)
            if not data:
                break
            f.write(data)
            receivedSize += len(data)
            print(f"Received {receivedSize}/{fileSize} bytes")

    print("File received successfully.")
    clientSocket.close()


TCP Server is listening...
Connected to ('127.0.0.1', 49332)
Received message: hello
Expecting file of size: 0 bytes
File received successfully.


In [ ]:
import socket
import threading

clients = []

def handleClient(clientSocket, clientAddress):
    while True:
        try:
            message = clientSocket.recv(1024)
            if not message:
                break
            print(f"{clientAddress}: {message.decode()}")
            for client in clients:
                if client != clientSocket:
                    client.sendall(message)
        except:
            break

    clients.remove(clientSocket)
    clientSocket.close()

serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSocket.bind(('localhost', 65432))
serverSocket.listen(5)
print("Chat Server is listening...")

while True:
    clientSocket, clientAddress = serverSocket.accept()
    clients.append(clientSocket)
    threading.Thread(target=handleClient, args=(clientSocket, clientAddress)).start()


Chat Server is listening...
('127.0.0.1', 65169): ad
('127.0.0.1', 65169): asd
('127.0.0.1', 65239): helo
('127.0.0.1', 65239): there
('127.0.0.1', 65239): sdfksdfjsk


In [ ]:
import socket
import requests

def getWeatherData(lat, lon):
    apiUrl = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    response = requests.get(apiUrl)
    if response.status_code == 200:
        return response.json()["current_weather"]["temperature"]
    return None

def sendWeatherData():
    temperature = getWeatherData(51.47, -0.0363)
    message = f"Current temperature: {temperature}°C" if temperature is not None else "Failed to fetch weather data"

    clientSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientSocket.connect(('localhost', 65432))
    clientSocket.sendall(message.encode())
    clientSocket.close()

sendWeatherData()
